Pyspark Kütüphanesinin Kurulumu

In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=4d494e7be09d57991130e37054fe333f7eaabad0040a3469fdb4a1dab50316e4
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


Python'da programınızın çalışması sırasında üretilen uyarıları görmezden gelmenizi sağlar.

In [110]:
import warnings
warnings.filterwarnings('ignore')

 "Makine Öğrenimi" adında bir uygulama adıyla bir Spark oturumu oluşturuyoruz

In [111]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Makine Öğrenimi").getOrCreate()

CSV dosyasından veri okuyoruz ve bir DataFrame'e yüklüyoruz

In [112]:
df = spark.read.csv("Data.csv",header=True,inferSchema=True)

DataFrame'in yapısını ve veri tiplerini kontrol ediyoruz

In [113]:
df.show()

+--------+------+-------+----------+------------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
| Loan_ID|Gender|Married|Dependents|   Education|Self_Employed|ApplicantIncome|CoapplicantIncome|LoanAmount|Loan_Amount_Term|Credit_History|Property_Area|Loan_Status|
+--------+------+-------+----------+------------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
|LP001002|  Male|     No|         0|    Graduate|           No|           5849|              0.0|      NULL|             360|             1|        Urban|          Y|
|LP001003|  Male|    Yes|         1|    Graduate|           No|           4583|           1508.0|       128|             360|             1|        Rural|          N|
|LP001005|  Male|    Yes|         0|    Graduate|          Yes|           3000|              0.0|        66|             360|             1|        Urban|          Y

In [114]:
df.printSchema()

root
 |-- Loan_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Married: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- Self_Employed: string (nullable = true)
 |-- ApplicantIncome: integer (nullable = true)
 |-- CoapplicantIncome: double (nullable = true)
 |-- LoanAmount: integer (nullable = true)
 |-- Loan_Amount_Term: integer (nullable = true)
 |-- Credit_History: integer (nullable = true)
 |-- Property_Area: string (nullable = true)
 |-- Loan_Status: string (nullable = true)



In [115]:
df.dtypes

[('Loan_ID', 'string'),
 ('Gender', 'string'),
 ('Married', 'string'),
 ('Dependents', 'string'),
 ('Education', 'string'),
 ('Self_Employed', 'string'),
 ('ApplicantIncome', 'int'),
 ('CoapplicantIncome', 'double'),
 ('LoanAmount', 'int'),
 ('Loan_Amount_Term', 'int'),
 ('Credit_History', 'int'),
 ('Property_Area', 'string'),
 ('Loan_Status', 'string')]

Burada DataFrame'deki sütun adlarını değiştiriyoruz

In [116]:
df = df.withColumnRenamed('Loan_ID', 'Id') \
       .withColumnRenamed('Gender', 'Cinsiyet') \
       .withColumnRenamed('Married', 'Evli') \
       .withColumnRenamed('Dependents', 'Bakımlılar') \
       .withColumnRenamed('Education', 'Eğitim') \
       .withColumnRenamed('Self_Employed', 'Serbest Meslek') \
       .withColumnRenamed('ApplicantIncome', 'Başvuru Sahibinin Geliri') \
       .withColumnRenamed('CoapplicantIncome', 'Ortak Başvuru Sahibinin Geliri') \
       .withColumnRenamed('LoanAmount', 'Kredi miktarı') \
       .withColumnRenamed('Loan_Amount_Term', 'Kredi Tutarı Vadesi') \
       .withColumnRenamed('Credit_History', 'Kredi geçmişi') \
       .withColumnRenamed('Property_Area', 'Mülk_Alan') \
       .withColumnRenamed('Loan_Status','Kredi_Durumu')

In [117]:
df.show()

+--------+--------+----+----------+------------+--------------+------------------------+------------------------------+-------------+-------------------+-------------+---------+------------+
|      Id|Cinsiyet|Evli|Bakımlılar|      Eğitim|Serbest Meslek|Başvuru Sahibinin Geliri|Ortak Başvuru Sahibinin Geliri|Kredi miktarı|Kredi Tutarı Vadesi|Kredi geçmişi|Mülk_Alan|Kredi_Durumu|
+--------+--------+----+----------+------------+--------------+------------------------+------------------------------+-------------+-------------------+-------------+---------+------------+
|LP001002|    Male|  No|         0|    Graduate|            No|                    5849|                           0.0|         NULL|                360|            1|    Urban|           Y|
|LP001003|    Male| Yes|         1|    Graduate|            No|                    4583|                        1508.0|          128|                360|            1|    Rural|           N|
|LP001005|    Male| Yes|         0|    Gradua

'Kredi_Durumu' sütununa göre DataFrame'i gruplar ve her grup için kayıt sayısını hesaplar

In [118]:
df.groupBy('Kredi_Durumu').count().show()

+------------+-----+
|Kredi_Durumu|count|
+------------+-----+
|           Y|  422|
|           N|  192|
+------------+-----+



In [119]:
import pyspark.sql.functions as Sp
import pandas as pd

B'Kredi geçmişi' ve 'Kredi_Durumu' sütunlarını seçeriz, sonra 'Kredi_Durumu' sütununa göre gruplar ve her grup için 'Kredi geçmişi' sütununun ortalamasını hesaplarız

In [120]:
dfn = df.select('Kredi geçmişi','Kredi_Durumu').groupBy('Kredi_Durumu').agg(Sp.avg('Kredi geçmişi')).show()

+------------+------------------+
|Kredi_Durumu|avg(Kredi geçmişi)|
+------------+------------------+
|           Y|0.9818181818181818|
|           N|0.5418994413407822|
+------------+------------------+



Bu kod,Sütunlar arasındaki korelasyonu hesaplayarak bir korelasyon matrisi oluşturur. Korelasyon matrisi, DataFrame'deki her iki sayısal değişken arasındaki ilişkiyi gösterir

In [121]:
columns = ['Başvuru Sahibinin Geliri','Ortak Başvuru Sahibinin Geliri','Kredi miktarı','Kredi Tutarı Vadesi', 'Kredi geçmişi']
corr_df = pd.DataFrame()
for i in columns:
    corr = []
    for j in columns:
        corr.append(round(df.corr(i, j), 2))
    corr_df = pd.concat([corr_df, pd.Series(corr)], axis=1)
corr_df.columns = columns
corr_df.insert(0,'' ,columns)
corr_df.set_index('')

,Başvuru Sahibinin Geliri,Ortak Başvuru Sahibinin Geliri,Kredi miktarı,Kredi Tutarı Vadesi,Kredi geçmişi
,,,,,
Başvuru Sahibinin Geliri,1.00,-0.12,0.54,-0.02,0.01
Ortak Başvuru Sahibinin Geliri,-0.12,1.00,0.19,-0.05,-0.06
Kredi miktarı,0.54,0.19,1.00,0.06,-0.03
Kredi Tutarı Vadesi,-0.02,-0.05,0.06,1.00,0.05
Kredi geçmişi,0.01,-0.06,-0.03,0.05,1.00


DataFrame içindeki her sütun için eksik değerlerinin sayısını hesapladık

In [122]:
df.select([Sp.count(Sp.when(Sp.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---+--------+----+----------+------+--------------+------------------------+------------------------------+-------------+-------------------+-------------+---------+------------+
| Id|Cinsiyet|Evli|Bakımlılar|Eğitim|Serbest Meslek|Başvuru Sahibinin Geliri|Ortak Başvuru Sahibinin Geliri|Kredi miktarı|Kredi Tutarı Vadesi|Kredi geçmişi|Mülk_Alan|Kredi_Durumu|
+---+--------+----+----------+------+--------------+------------------------+------------------------------+-------------+-------------------+-------------+---------+------------+
|  0|      13|   3|        15|     0|            32|                       0|                             0|           22|                 14|           50|        0|           0|
+---+--------+----+----------+------+--------------+------------------------+------------------------------+-------------+-------------------+-------------+---------+------------+



'Kredi miktarı' sütunundaki değerlerin ortalamasını hesapladık

In [123]:
mean = df.select(Sp.mean(df['Kredi miktarı'])).collect()[0][0]# collect()
mean

146.41216216216216

Eksik verileri doldurma işlemi yaptık

In [124]:
df = df.na.fill(mean, ['Kredi miktarı'])

Bu kod en çok tekrar eden cinsiyeti verir

In [125]:
df.groupby('Cinsiyet').count().orderBy("count", ascending=False).first()[0]

'Male'

In [126]:
numerical_cols = ['Kredi miktarı', 'Kredi Tutarı Vadesi']
categorical_cols = ['Cinsiyet', 'Evli', 'Bakımlılar', 'Serbest Meslek', 'Kredi geçmişi']

be sütunlardaki eksik değerleri bu sütunların ortalaması ile doldurdum

In [127]:
for col in numerical_cols:
    mean = df.select(Sp.mean(df[col])).collect()[0][0]
    df = df.na.fill(mean, [col])

In [128]:
for col in categorical_cols:
    mode = df.groupby(col).count().orderBy("count", ascending=False).first()[0]
    df = df.na.fill(mode, [col])

Eksik verim varmı diye kontrol ettim

In [129]:
df.select([Sp.count(Sp.when(Sp.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---+--------+----+----------+------+--------------+------------------------+------------------------------+-------------+-------------------+-------------+---------+------------+
| Id|Cinsiyet|Evli|Bakımlılar|Eğitim|Serbest Meslek|Başvuru Sahibinin Geliri|Ortak Başvuru Sahibinin Geliri|Kredi miktarı|Kredi Tutarı Vadesi|Kredi geçmişi|Mülk_Alan|Kredi_Durumu|
+---+--------+----+----------+------+--------------+------------------------+------------------------------+-------------+-------------------+-------------+---------+------------+
|  0|       0|   0|         0|     0|             0|                       0|                             0|            0|                  0|            0|        0|           0|
+---+--------+----+----------+------+--------------+------------------------+------------------------------+-------------+-------------------+-------------+---------+------------+



'Başvuru Sahibinin Geliri' ve 'Ortak Başvuru Sahibinin Geliri' sütunlarındaki değerlerin toplamını alarak yeni bir 'Toplam gelir' sütunu oluşturdum

In [130]:
df = df.withColumn('Toplam gelir', Sp.col('Başvuru Sahibinin Geliri') + Sp.col('Ortak Başvuru Sahibinin Geliri'))
df.show(5)

+--------+--------+----+----------+------------+--------------+------------------------+------------------------------+-------------+-------------------+-------------+---------+------------+------------+
|      Id|Cinsiyet|Evli|Bakımlılar|      Eğitim|Serbest Meslek|Başvuru Sahibinin Geliri|Ortak Başvuru Sahibinin Geliri|Kredi miktarı|Kredi Tutarı Vadesi|Kredi geçmişi|Mülk_Alan|Kredi_Durumu|Toplam gelir|
+--------+--------+----+----------+------------+--------------+------------------------+------------------------------+-------------+-------------------+-------------+---------+------------+------------+
|LP001002|    Male|  No|         0|    Graduate|            No|                    5849|                           0.0|          146|                360|            1|    Urban|           Y|      5849.0|
|LP001003|    Male| Yes|         1|    Graduate|            No|                    4583|                        1508.0|          128|                360|            1|    Rural|       

'Kredi_Durumu' sütununda 'Y' değeri varsa, bu değeri 1 olarak ayarlar, aksi halde 0 olarak ayarladım

In [131]:
df = df.withColumn('Kredi_Durumu', Sp.when(df['Kredi_Durumu']=='Y', 1).otherwise(0))
df.show(5)

+--------+--------+----+----------+------------+--------------+------------------------+------------------------------+-------------+-------------------+-------------+---------+------------+------------+
|      Id|Cinsiyet|Evli|Bakımlılar|      Eğitim|Serbest Meslek|Başvuru Sahibinin Geliri|Ortak Başvuru Sahibinin Geliri|Kredi miktarı|Kredi Tutarı Vadesi|Kredi geçmişi|Mülk_Alan|Kredi_Durumu|Toplam gelir|
+--------+--------+----+----------+------------+--------------+------------------------+------------------------------+-------------+-------------------+-------------+---------+------------+------------+
|LP001002|    Male|  No|         0|    Graduate|            No|                    5849|                           0.0|          146|                360|            1|    Urban|           1|      5849.0|
|LP001003|    Male| Yes|         1|    Graduate|            No|                    4583|                        1508.0|          128|                360|            1|    Rural|       

In [132]:
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Cinsiyet: string (nullable = false)
 |-- Evli: string (nullable = false)
 |-- Bakımlılar: string (nullable = false)
 |-- Eğitim: string (nullable = true)
 |-- Serbest Meslek: string (nullable = false)
 |-- Başvuru Sahibinin Geliri: integer (nullable = true)
 |-- Ortak Başvuru Sahibinin Geliri: double (nullable = true)
 |-- Kredi miktarı: integer (nullable = true)
 |-- Kredi Tutarı Vadesi: integer (nullable = true)
 |-- Kredi geçmişi: integer (nullable = true)
 |-- Mülk_Alan: string (nullable = true)
 |-- Kredi_Durumu: integer (nullable = false)
 |-- Toplam gelir: double (nullable = true)



In [133]:
from pyspark.ml.feature import VectorAssembler, OneHotEncoder, StringIndexer
from pyspark.ml import Pipeline

In [134]:
categorical_columns = ['Cinsiyet', 'Evli', 'Bakımlılar', 'Eğitim', 'Serbest Meslek', 'Mülk_Alan', 'Kredi geçmişi']
numerical_columns = ['Başvuru Sahibinin Geliri', 'Ortak Başvuru Sahibinin Geliri', 'Kredi miktarı', 'Kredi Tutarı Vadesi', 'Toplam gelir']

 kategorik sütunların dizinlenmesi, ardından ikili vektör haline getirilmesi ve sayısal sütunlarla birleştirilerek bir özellik vektörü oluşturulması için gerekli işlemleri gerçekleştirdim

In [135]:
indexers = [StringIndexer(inputCol=col, outputCol="{0}_index".format(col)) for col in categorical_columns]#outputCol="{0}_index".format(col) bu yeni sütun adını belirler.
encoders = [OneHotEncoder(dropLast=False, inputCol=indexer.getOutputCol(), outputCol="{0}_encoded".format(indexer.getOutputCol())) for indexer in indexers]#getOutputCol  outputCol'dan ismi cekern bu kadar.
input_columns = [encoder.getOutputCol() for encoder in encoders] + numerical_columns
assembler = VectorAssembler(inputCols=input_columns, outputCol="feature")

pipeline oluşturdum

In [136]:
pipeline = Pipeline(stages = indexers + encoders + [assembler])

makine öğrenimi modeli için DataFrame'i hazırlıyoruz

In [137]:
data_model = pipeline.fit(df)

In [138]:
transformed_df = data_model.transform(df)
transformed_df.show(1)

+--------+--------+----+----------+--------+--------------+------------------------+------------------------------+-------------+-------------------+-------------+---------+------------+------------+--------------+----------+----------------+------------+--------------------+---------------+-------------------+----------------------+------------------+------------------------+--------------------+----------------------------+-----------------------+---------------------------+--------------------+
|      Id|Cinsiyet|Evli|Bakımlılar|  Eğitim|Serbest Meslek|Başvuru Sahibinin Geliri|Ortak Başvuru Sahibinin Geliri|Kredi miktarı|Kredi Tutarı Vadesi|Kredi geçmişi|Mülk_Alan|Kredi_Durumu|Toplam gelir|Cinsiyet_index|Evli_index|Bakımlılar_index|Eğitim_index|Serbest Meslek_index|Mülk_Alan_index|Kredi geçmişi_index|Cinsiyet_index_encoded|Evli_index_encoded|Bakımlılar_index_encoded|Eğitim_index_encoded|Serbest Meslek_index_encoded|Mülk_Alan_index_encoded|Kredi geçmişi_index_encoded|             fea

In [139]:
transformed_df = transformed_df.select(['feature', 'Kredi_Durumu'])
train_data, test_data = transformed_df.randomSplit([0.8, 0.2], seed=42)
train_data.show(5)

+--------------------+------------+
|             feature|Kredi_Durumu|
+--------------------+------------+
|(22,[0,2,4,8,10,1...|           1|
|(22,[0,2,4,8,10,1...|           1|
|(22,[0,2,4,8,10,1...|           0|
|(22,[0,2,4,8,10,1...|           1|
|(22,[0,2,4,8,10,1...|           1|
+--------------------+------------+
only showing top 5 rows



In [140]:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [153]:
lr = LogisticRegression(featuresCol='feature', labelCol='Kredi_Durumu')
lr_model = lr.fit(train_data)

In [154]:
predictions = lr_model.transform(test_data)
predictions.show(5)

+--------------------+------------+--------------------+--------------------+----------+
|             feature|Kredi_Durumu|       rawPrediction|         probability|prediction|
+--------------------+------------+--------------------+--------------------+----------+
|(22,[0,2,4,8,10,1...|           1|[-2.1309019654107...|[0.10612939546952...|       1.0|
|(22,[0,2,4,8,10,1...|           1|[-2.1965880165663...|[0.10005730505861...|       1.0|
|(22,[0,2,4,8,10,1...|           1|[-2.1183870711017...|[0.10732249829704...|       1.0|
|(22,[0,2,4,8,10,1...|           1|[-2.1420673697694...|[0.10507482746799...|       1.0|
|(22,[0,2,4,8,10,1...|           1|[-2.1120272450757...|[0.10793332086955...|       1.0|
+--------------------+------------+--------------------+--------------------+----------+
only showing top 5 rows



BinaryClassificationEvaluator yardımı ile Logistic Regression ve RandomForestClassifier performans ölcümü yaptım.

In [155]:
predictions = lr_model.transform(test_data)
auc = BinaryClassificationEvaluator().setLabelCol('Kredi_Durumu')
print('AUC:', str(auc.evaluate(predictions)))

AUC: 0.782010582010582


In [144]:
rf = RandomForestClassifier(featuresCol='feature', labelCol='Kredi_Durumu')
rf_model = rf.fit(train_data)

In [145]:
predictions = rf_model.transform(test_data)
auc = BinaryClassificationEvaluator().setLabelCol('Kredi_Durumu')
print('AUC:', str(auc.evaluate(predictions)))

AUC: 0.8227513227513228


DataFrame'deki verileri Pandas DataFrame'ine dönüştürüp gösterme işlemi gerçekleştiyiyorum

In [156]:
df_pandas = df.toPandas()
df_pandas.head()

,Id,Cinsiyet,Evli,Bakımlılar,Eğitim,Serbest Meslek,Başvuru Sahibinin Geliri,Ortak Başvuru Sahibinin Geliri,Kredi miktarı,Kredi Tutarı Vadesi,Kredi geçmişi,Mülk_Alan,Kredi_Durumu,Toplam gelir
0,LP001002,Male,No,0,Graduate,No,5849,0.0,146,360,1,Urban,1,5849.0
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128,360,1,Rural,0,6091.0
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66,360,1,Urban,1,3000.0
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120,360,1,Urban,1,4941.0
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141,360,1,Urban,1,6000.0
